# BigQuery のテーブル定義を抽出

## ライブラリのインポートと対象データセットの設定

In [1]:
import pandas as pd
from google.cloud import bigquery

In [2]:
# 対象データセットの設定
PROJECT_ID = "bigquery-public-data"
DATASET_ID = "chicago_taxi_trips"

In [3]:
# BigQueryクライアントのインスタンスを生成
client = bigquery.Client()

## メタデータの取得

次に、テーブル定義書に必要なメタデータを、INFORMATION_SCHEMA から取得していきます。

まずは、INFORMATION_SCHEMA.COLUMNS ビューからメタデータを取得します。

In [4]:
query = f"""
    SELECT
      *
    FROM
      `{PROJECT_ID}.{DATASET_ID}.INFORMATION_SCHEMA.COLUMNS`;
"""

# クエリを実行してDataFrameとして取得
df = client.query(query).to_dataframe()
df.head()

,table_catalog,table_schema,table_name,column_name,ordinal_position,is_nullable,data_type,is_generated,generation_expression,is_stored,is_hidden,is_updatable,is_system_defined,is_partitioning_column,clustering_ordinal_position,collation_name,column_default,rounding_mode
0,bigquery-public-data,chicago_taxi_trips,taxi_trips,unique_key,1,NO,STRING,NEVER,None,None,NO,None,NO,NO,<NA>,NULL,NULL,None
1,bigquery-public-data,chicago_taxi_trips,taxi_trips,taxi_id,2,NO,STRING,NEVER,None,None,NO,None,NO,NO,<NA>,NULL,NULL,None
2,bigquery-public-data,chicago_taxi_trips,taxi_trips,trip_start_timestamp,3,YES,TIMESTAMP,NEVER,None,None,NO,None,NO,NO,<NA>,NULL,NULL,None
3,bigquery-public-data,chicago_taxi_trips,taxi_trips,trip_end_timestamp,4,YES,TIMESTAMP,NEVER,None,None,NO,None,NO,NO,<NA>,NULL,NULL,None
4,bigquery-public-data,chicago_taxi_trips,taxi_trips,trip_seconds,5,YES,INT64,NEVER,None,None,NO,None,NO,NO,<NA>,NULL,NULL,None


テーブル定義書に必要な列を抽出します。

In [5]:
df_columns = df[
    [
        "table_catalog",
        "table_schema",
        "table_name",
        "column_name",
        "ordinal_position",
        "data_type",
        "is_nullable",
        "is_partitioning_column",
        "clustering_ordinal_position",
    ]
]

# Excel書き込み時のNA対策でstr型に変換
df_columns = df_columns.astype({"clustering_ordinal_position": str})
df_columns.head()

,table_catalog,table_schema,table_name,column_name,ordinal_position,data_type,is_nullable,is_partitioning_column,clustering_ordinal_position
0,bigquery-public-data,chicago_taxi_trips,taxi_trips,unique_key,1,STRING,NO,NO,<NA>
1,bigquery-public-data,chicago_taxi_trips,taxi_trips,taxi_id,2,STRING,NO,NO,<NA>
2,bigquery-public-data,chicago_taxi_trips,taxi_trips,trip_start_timestamp,3,TIMESTAMP,YES,NO,<NA>
3,bigquery-public-data,chicago_taxi_trips,taxi_trips,trip_end_timestamp,4,TIMESTAMP,YES,NO,<NA>
4,bigquery-public-data,chicago_taxi_trips,taxi_trips,trip_seconds,5,INT64,YES,NO,<NA>


次に、INFORMATION_SCHEMA.COLUMN_FIELD_PATHS ビューからメタデータを取得します。

In [6]:
query = f"""
    SELECT
      *
    FROM
      `{PROJECT_ID}.{DATASET_ID}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`;
"""

# クエリを実行してDataFrameとして取得
df = client.query(query).to_dataframe()
df.head()

,table_catalog,table_schema,table_name,column_name,field_path,data_type,description,collation_name,rounding_mode
0,bigquery-public-data,chicago_taxi_trips,taxi_trips,unique_key,unique_key,STRING,Unique identifier for the trip.,NULL,None
1,bigquery-public-data,chicago_taxi_trips,taxi_trips,taxi_id,taxi_id,STRING,A unique identifier for the taxi.,NULL,None
2,bigquery-public-data,chicago_taxi_trips,taxi_trips,trip_start_timestamp,trip_start_timestamp,TIMESTAMP,"When the trip started, rounded to the nearest ...",NULL,None
3,bigquery-public-data,chicago_taxi_trips,taxi_trips,trip_end_timestamp,trip_end_timestamp,TIMESTAMP,"When the trip ended, rounded to the nearest 15...",NULL,None
4,bigquery-public-data,chicago_taxi_trips,taxi_trips,trip_seconds,trip_seconds,INT64,Time of the trip in seconds.,NULL,None


テーブル定義書に必要な列を抽出します。

In [7]:
df_description = df[["table_name", "column_name", "description"]]
df_description.head()

,table_name,column_name,description
0,taxi_trips,unique_key,Unique identifier for the trip.
1,taxi_trips,taxi_id,A unique identifier for the taxi.
2,taxi_trips,trip_start_timestamp,"When the trip started, rounded to the nearest ..."
3,taxi_trips,trip_end_timestamp,"When the trip ended, rounded to the nearest 15..."
4,taxi_trips,trip_seconds,Time of the trip in seconds.


最後に、取得した df_columns と df_description をマージします。

ここで、レコードを一意に特定するため、table_name と column_name の2つのキーを使用しています。

In [8]:
df_table_def = df_columns.merge(df_description, on=["table_name", "column_name"])
df_table_def.head()

,table_catalog,table_schema,table_name,column_name,ordinal_position,data_type,is_nullable,is_partitioning_column,clustering_ordinal_position,description
0,bigquery-public-data,chicago_taxi_trips,taxi_trips,unique_key,1,STRING,NO,NO,<NA>,Unique identifier for the trip.
1,bigquery-public-data,chicago_taxi_trips,taxi_trips,taxi_id,2,STRING,NO,NO,<NA>,A unique identifier for the taxi.
2,bigquery-public-data,chicago_taxi_trips,taxi_trips,trip_start_timestamp,3,TIMESTAMP,YES,NO,<NA>,"When the trip started, rounded to the nearest ..."
3,bigquery-public-data,chicago_taxi_trips,taxi_trips,trip_end_timestamp,4,TIMESTAMP,YES,NO,<NA>,"When the trip ended, rounded to the nearest 15..."
4,bigquery-public-data,chicago_taxi_trips,taxi_trips,trip_seconds,5,INT64,YES,NO,<NA>,Time of the trip in seconds.


テーブル定義書に必要なメタデータをデータフレームとして抽出することができました。

# エクセルに出力

前準備として、テンプレートファイルを作業ディレクトリに保存します。

ノートブックでは、次のコマンドを実行して、GitHubからテンプレートファイルをダウンロードします。

In [ ]:
!wget https://github.com/cloud-ace/zenn-bq-table-definition/raw/main/table_template.xlsx

Python で Excel ファイルを操作するために、 openpyxl をインポートします。

また、メタデータを転記するセルの開始位置を指定します。

In [10]:
from openpyxl import load_workbook

# テンプレートファイルの設定
STRT_TBL = (2, 3) # テーブル情報の開始位置 (行, 列)
STRT_COL = (8, 1) # カラム情報の開始位置 (行, 列)

In [11]:
# テンプレートファイルを開く
wb = load_workbook("./table_template.xlsx")

# templateシートを取得
ws_template = wb["template"]

# tableごとにテーブル定義書を作成
for table_name, df in df_table_def.groupby("table_name"):

    # templeteシートをコピペ
    ws = wb.copy_worksheet(ws_template)
    print(f"Table Name: {table_name}")

    # シート名をテーブル名に変更
    ws.title = table_name

    # テーブル情報の書き込み
    ws.cell(
        row=STRT_TBL[0],
        column=STRT_TBL[1],
        value=df["table_catalog"].iloc[0],
    )

    ws.cell(
        row=STRT_TBL[0] + 1,
        column=STRT_TBL[1],
        value=df["table_schema"].iloc[0],
    )

    ws.cell(
        row=STRT_TBL[0] + 2,
        column=STRT_TBL[1],
        value=df["table_name"].iloc[0],
    )

    # カラム情報の書き込み
    for i, (_, sr) in enumerate(df.iterrows()):
        row = STRT_COL[0] + i

        ws.cell(row=row, column=STRT_COL[1], value=sr["ordinal_position"])
        ws.cell(row=row, column=STRT_COL[1] + 1, value=sr["column_name"])
        ws.cell(row=row, column=STRT_COL[1] + 2, value=sr["data_type"])
        ws.cell(row=row, column=STRT_COL[1] + 3, value=sr["is_nullable"])
        ws.cell(row=row, column=STRT_COL[1] + 4, value=sr["is_partitioning_column"])
        ws.cell(row=row, column=STRT_COL[1] + 5, value=sr["clustering_ordinal_position"])
        ws.cell(row=row, column=STRT_COL[1] + 6, value=sr["description"])

# templeteシートを削除
wb.remove(ws_template)

# テーブル定義書（エクセルファイル）の保存
save_path = f"./table_definition_{PROJECT_ID}.{DATASET_ID}.xlsx"
wb.save(save_path)
print(f"Saved file at {save_path}")

Table Name: taxi_trips
Saved file at ./table_definition_bigquery-public-data.chicago_taxi_trips.xlsx


テーブル定義書のエクセルファイルが出力されるので、ダウンロードします。

以上です。